[![pythonista](img/pythonista.png)](https://www.pythonista.io)

# *SQLAlchemy* asíncrono.

*SQLAlchemy* cuenta con una extensión compatible con ```asyncio``` mediante una extensión especializada.

https://docs.sqlalchemy.org/en/latest/orm/extensions/asyncio.html

In [ ]:
!pip install sqlalchemy[asyncio] aiosqlite

In [ ]:
!rm db.sqlite3

In [ ]:
from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy.ext.asyncio import create_async_engine

In [ ]:
engine = create_async_engine('sqlite+aiosqlite:///db.sqlite3')

In [ ]:
import sqlalchemy as sa
from sqlalchemy.orm import declarative_base, sessionmaker

In [ ]:
Base = declarative_base()

In [ ]:
class Alumno(Base):
    __tablename__ = 'alumnos'
    cuenta = sa.Column(sa.Integer, primary_key=True)
    nombre = sa.Column(sa.String(50))
    primer_apellido = sa.Column(sa.String(50))
    segundo_apellido = sa.Column(sa.String(50))
    carrera = sa.Column(sa.String(50))
    semestre = sa.Column(sa.Integer)
    promedio = sa.Column(sa.Float)
    al_corriente = sa.Column(sa.Boolean)

In [ ]:
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.create_all)

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///db.sqlite3

In [ ]:
%%sql 
select * from alumnos

In [ ]:
async_session = sessionmaker(engine, 
                                 expire_on_commit=False, 
                                 class_=AsyncSession)

In [ ]:
 async with async_session() as session:
    with open('data/alumnos.py') as f:
        for alumno in eval(f.read()):
            async with session.begin():
                obj = Alumno(**alumno)
                session.add(obj)

In [ ]:
%%sql 
select * from alumnos

In [ ]:
async with async_session() as session:
    async with session.begin():
        alumnos = await session.execute(sa.select(Alumno))
        for alumno in alumnos.scalars():
            print(alumno.cuenta, alumno.nombre, alumno.primer_apellido)

In [ ]:
async with async_session() as session:
    async with session.begin():
        stmnt = sa.select(Alumno).where(Alumno.nombre == "Pedro")
        alumnos = await session.execute(stmnt)
        for alumno in alumnos.scalars():
            print(alumno.cuenta, alumno.nombre, alumno.primer_apellido)

In [ ]:
async with async_session() as session:
    async with session.begin():
        stmnt = sa.delete(Alumno).where(Alumno.cuenta == 1231221)
        await session.execute(stmnt)

In [ ]:
%%sql 
select * from alumnos

In [ ]:
async with async_session() as session:
    async with session.begin():
        stmnt = sa.update(Alumno).where(Alumno.cuenta == 1231222).values(nombre="Yola")
        await session.execute(stmnt)

In [ ]:
%%sql 
select * from alumnos

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>